# Excel Flattener - Testing Notebook

Interactive notebook for testing and exploring the Excel Flattener.

## Features Tested

- Workbook flattening
- Formula extraction
- Value extraction (literal and computed)
- VBA extraction
- Chart, table, and named range extraction
- Manifest generation

## Setup

Ensure dependencies are installed:
```bash
pip install -r requirements.txt
pip install jupyter
```

## Configuration

**Edit this cell to customise settings for testing:**

In [9]:
# ===== CONFIGURATION =====
# Edit these variables to customise your test run

# Test file to flatten
TEST_EXCEL_FILE = "./sample.xlsm"  # Change to your test file
TEST_EXCEL_FILE = "./sample.xlsx"  # Change to your test file

# Output directory
OUTPUT_DIR = "./tmp/flats"

# Include computed values?
INCLUDE_COMPUTED = True

# Logging level (DEBUG, INFO, WARNING, ERROR)
LOG_LEVEL = "INFO"

# Extraction timeout (seconds)
EXTRACTION_TIMEOUT = 900

# Maximum file size (MB)
MAX_FILE_SIZE_MB = 200

# Test for Manifest
ORIGIN_REPO = "Origin repo test"
ORIGIN_PATH = "Origin path test"
ORIGIN_COMMIT = "Origin commit test"
ORIGIN_COMMIT_MESSAGE = "Origin commit message test"

# =========================

## Imports and Setup

In [10]:
import sys
from pathlib import Path
import json
from datetime import datetime

# Add parent directory to path to import from src
sys.path.insert(0, str(Path('..').resolve()))

from src import Flattener, setup_logging
from src.utils import get_file_hash

# Setup logging
setup_logging(log_level=LOG_LEVEL, log_dir='./tmp/logs', component='notebook')

print("✓ Imports complete")
print(f"✓ Test file: {TEST_EXCEL_FILE}")
print(f"✓ Output directory: {OUTPUT_DIR}")

[2025-10-30 11:31:52] INFO     | root         | Logging initialised (level: INFO, file: tmp/logs/notebook_20251030_113152.log)
✓ Imports complete
✓ Test file: ./sample.xlsx
✓ Output directory: ./tmp/flats


## Validate Test File

In [11]:
test_file = Path(TEST_EXCEL_FILE)

if not test_file.exists():
    print(f"✗ Test file not found: {test_file}")
    print("\nPlease create a test Excel file or update TEST_EXCEL_FILE in the config cell.")
else:
    # File information
    size_bytes = test_file.stat().st_size
    size_mb = size_bytes / (1024 * 1024)
    
    print(f"✓ Test file found")
    print(f"  Name: {test_file.name}")
    print(f"  Size: {size_mb:.2f} MB ({size_bytes:,} bytes)")
    print(f"  Extension: {test_file.suffix}")
    
    # Calculate hash
    print(f"\n  Computing SHA256...")
    file_hash = get_file_hash(test_file)
    print(f"  SHA256: {file_hash}")
    print(f"  Short hash: {file_hash[:16]}...")

✓ Test file found
  Name: sample.xlsx
  Size: 0.01 MB (8,658 bytes)
  Extension: .xlsx

  Computing SHA256...
  SHA256: 4f4cb6ba08415d0de58f50d614e08b7effee0975ba4fa11dd8782d1e38181c86
  Short hash: 4f4cb6ba08415d0d...


## Create Flattener Instance

In [12]:
flattener = Flattener(
    output_dir=Path(OUTPUT_DIR),
    include_computed=INCLUDE_COMPUTED,
    timeout=EXTRACTION_TIMEOUT,
    max_file_size_mb=MAX_FILE_SIZE_MB
)

print("✓ Flattener created")
print(f"  Output directory: {OUTPUT_DIR}")
print(f"  Include computed: {INCLUDE_COMPUTED}")
print(f"  Timeout: {EXTRACTION_TIMEOUT}s")
print(f"  Max file size: {MAX_FILE_SIZE_MB}MB")

[2025-10-30 11:32:08] INFO     | src.flattener | Flattener initialised (output: tmp/flats, computed: True, timeout: 900s)
✓ Flattener created
  Output directory: ./tmp/flats
  Include computed: True
  Timeout: 900s
  Max file size: 200MB


## Flatten Workbook

In [13]:
start_time = datetime.now()

try:
    flat_root = flattener.flatten(
        excel_file=test_file,
        origin_repo=ORIGIN_REPO,
        origin_path=ORIGIN_PATH,
        origin_commit=ORIGIN_COMMIT,
        origin_commit_message=ORIGIN_COMMIT_MESSAGE
    )
    
    end_time = datetime.now()
    duration = (end_time - start_time).total_seconds()
    
    print(f"\n✓ Flattening complete!")
    print(f"  Duration: {duration:.2f}s")
    print(f"  Output: {flat_root}")
    
except Exception as e:
    end_time = datetime.now()
    duration = (end_time - start_time).total_seconds()
    
    print(f"\n✗ Flattening failed after {duration:.2f}s")
    print(f"  Error: {e}")
    raise

[2025-10-30 11:32:12] INFO     | src.flattener | ✓ File validated (0.0MB)
[2025-10-30 11:32:12] INFO     | src.flattener | ======================================================================
[2025-10-30 11:32:12] INFO     | src.flattener | Starting extraction: sample.xlsx
[2025-10-30 11:32:12] INFO     | src.flattener | ======================================================================
[2025-10-30 11:32:12] INFO     | src.flattener | File hash: 4f4cb6ba08415d0d...
[2025-10-30 11:32:12] INFO     | src.flattener | Flat root: tmp/flats/sample-flat-20251030T113212Z-4f4cb6ba
[2025-10-30 11:32:12] INFO     | src.flattener | Loading workbook...
[2025-10-30 11:32:12] INFO     | src.flattener | ✓ Workbook loaded (3 sheets)
[2025-10-30 11:32:12] INFO     | src.flattener | Extracting metadata...
[2025-10-30 11:32:12] INFO     | src.metadata | ✓ Extracted metadata (author: Excel Flattener Test Suite, sheets: 3)
[2025-10-30 11:32:12] INFO     | src.flattener | Extracting structure...
[2025-1

## Examine Manifest

In [ ]:
manifest_path = flat_root / 'manifest.json'

with open(manifest_path, 'r', encoding='utf-8') as f:
    manifest = json.load(f)

print("Manifest Overview:")
print("=" * 50)
print(f"Workbook: {manifest['workbook_filename']}")
print(f"Extracted at: {manifest['extracted_at']}")
print(f"Extractor version: {manifest['extractor_version']}")
print(f"Include computed: {manifest['include_computed']}")
print(f"\nOriginal file hash: {manifest['original_sha256'][:16]}...")

print(f"\nSheets ({len(manifest['sheets'])}):")
for sheet in manifest['sheets']:
    visibility = '👁️ ' if sheet['visible'] else '🔒 '
    print(f"  {visibility}{sheet['index']}. {sheet['name']} (ID: {sheet['sheetId']})")

print(f"\nFiles generated: {len(manifest['files'])}")
for file_info in manifest['files'][:10]:  # Show first 10
    print(f"  - {file_info['path']}")
if len(manifest['files']) > 10:
    print(f"  ... and {len(manifest['files']) - 10} more")

if manifest['warnings']:
    print(f"\n⚠️  Warnings ({len(manifest['warnings'])}):")
    for warning in manifest['warnings']:
        print(f"  - {warning}")
else:
    print(f"\n✓ No warnings")

if 'origin' in manifest:
    origin = manifest['origin']
    print(f"\nOrigin:")
    print(f"  Repo: {origin['origin_repo']}")
    print(f"  Path: {origin['origin_path']}")
    print(f"  Commit: {origin['origin_commit']}")

## Examine Metadata

In [ ]:
metadata_path = flat_root / 'metadata.txt'

if metadata_path.exists():
    print("Workbook Metadata:")
    print("=" * 50)
    with open(metadata_path, 'r', encoding='utf-8') as f:
        # Skip header
        lines = f.readlines()[3:]
        for line in lines:
            if line.strip():
                print(f"  {line.rstrip()}")
else:
    print("No metadata file found")

## Examine Structure

In [ ]:
structure_path = flat_root / 'structure.txt'

if structure_path.exists():
    print("Workbook Structure:")
    print("=" * 50)
    with open(structure_path, 'r', encoding='utf-8') as f:
        # Skip header
        lines = f.readlines()[3:]
        print(''.join(lines))
else:
    print("No structure file found")

## Examine Sheet Data (First Sheet)

In [ ]:
sheets_dir = flat_root / 'sheets'

if sheets_dir.exists():
    # Get first sheet directory
    sheet_dirs = sorted(sheets_dir.iterdir())
    if sheet_dirs:
        first_sheet = sheet_dirs[0]
        print(f"Sheet: {first_sheet.name}")
        print("=" * 50)
        
        # Formulas
        formulas_path = first_sheet / 'formulas.txt'
        if formulas_path.exists():
            with open(formulas_path, 'r', encoding='utf-8') as f:
                lines = f.readlines()[3:]  # Skip header
                formula_count = len([l for l in lines if l.strip()])
                print(f"\nFormulas ({formula_count}):")
                for line in lines[:5]:  # Show first 5
                    if line.strip():
                        print(f"  {line.rstrip()}")
                if formula_count > 5:
                    print(f"  ... and {formula_count - 5} more")
        
        # Literal values
        literal_path = first_sheet / 'literal-values.txt'
        if literal_path.exists():
            with open(literal_path, 'r', encoding='utf-8') as f:
                lines = f.readlines()[3:]  # Skip header
                value_count = len([l for l in lines if l.strip()])
                print(f"\nLiteral Values ({value_count}):")
                for line in lines[:5]:  # Show first 5
                    if line.strip():
                        print(f"  {line.rstrip()}")
                if value_count > 5:
                    print(f"  ... and {value_count - 5} more")
        
        # Computed values
        computed_path = first_sheet / 'computed-values.txt'
        if computed_path.exists():
            with open(computed_path, 'r', encoding='utf-8') as f:
                lines = f.readlines()[3:]  # Skip header
                value_count = len([l for l in lines if l.strip()])
                print(f"\nComputed Values ({value_count}):")
                for line in lines[:5]:  # Show first 5
                    if line.strip():
                        print(f"  {line.rstrip()}")
                if value_count > 5:
                    print(f"  ... and {value_count - 5} more")
    else:
        print("No sheets found")
else:
    print("No sheets directory found")

## Examine VBA (if present)

In [ ]:
vba_dir = flat_root / 'vba'

if vba_dir.exists():
    print("VBA Macros:")
    print("=" * 50)
    
    # Summary
    summary_path = vba_dir / 'vba-summary.txt'
    if summary_path.exists():
        with open(summary_path, 'r', encoding='utf-8') as f:
            print(f.read())
    
    # List VBA files
    vba_files = list(vba_dir.glob('*.vba'))
    if vba_files:
        print(f"\nVBA Modules ({len(vba_files)}):")
        for vba_file in vba_files:
            with open(vba_file, 'r', encoding='utf-8', errors='replace') as f:
                lines = f.readlines()
                line_count = len(lines)
                print(f"  - {vba_file.name} ({line_count} lines)")
else:
    print("No VBA macros found")

## Examine Other Features

In [ ]:
# Named ranges
named_ranges_path = flat_root / 'named-ranges.txt'
if named_ranges_path.exists():
    with open(named_ranges_path, 'r', encoding='utf-8') as f:
        content = f.read()
        # Count names
        name_count = content.count('Name: ')
        print(f"Named Ranges: {name_count}")

# Tables
tables_path = flat_root / 'tables.txt'
if tables_path.exists():
    with open(tables_path, 'r', encoding='utf-8') as f:
        content = f.read()
        table_count = content.count('Table: ')
        print(f"Tables: {table_count}")

# AutoFilters
autofilters_path = flat_root / 'autofilters.txt'
if autofilters_path.exists():
    with open(autofilters_path, 'r', encoding='utf-8') as f:
        content = f.read()
        filter_count = content.count('Sheet: ')
        print(f"AutoFilters: {filter_count}")

# Charts
charts_path = flat_root / 'charts.txt'
if charts_path.exists():
    with open(charts_path, 'r', encoding='utf-8') as f:
        content = f.read()
        chart_count = content.count('Chart ')
        print(f"Charts: {chart_count}")

## Directory Listing

In [ ]:
import os

def print_tree(directory, prefix='', max_depth=3, current_depth=0):
    """Print directory tree."""
    if current_depth >= max_depth:
        return
    
    entries = sorted(directory.iterdir(), key=lambda x: (not x.is_dir(), x.name))
    
    for i, entry in enumerate(entries):
        is_last = i == len(entries) - 1
        current_prefix = '└── ' if is_last else '├── '
        print(f"{prefix}{current_prefix}{entry.name}")
        
        if entry.is_dir():
            extension = '    ' if is_last else '│   '
            print_tree(entry, prefix + extension, max_depth, current_depth + 1)

print("\nOutput Directory Tree:")
print("=" * 50)
print(flat_root.name)
print_tree(flat_root, max_depth=3)

## Summary

In [ ]:
print("\nExtraction Summary:")
print("=" * 50)
print(f"Input file: {test_file}")
print(f"Output directory: {flat_root}")
print(f"Duration: {duration:.2f}s")
print(f"Files generated: {len(manifest['files'])}")
print(f"Sheets: {len(manifest['sheets'])}")
print(f"Warnings: {len(manifest['warnings'])}")
print(f"Include computed: {INCLUDE_COMPUTED}")
print("\n✓ Testing complete!")